In [2]:
import torch
torch.__version__

'2.4.0+cu121'

In [3]:
import torch
torch.cuda.is_available()

True

In [5]:
import torch
tensor0d = torch.tensor(1) #A
tensor1d = torch.tensor([1, 2, 3]) #B
tensor2d = torch.tensor([[1, 2], [3, 4]]) #C
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]]) #D

In [11]:
# tensor types
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

float_tensor = torch.tensor([1.0, 2.0, 3.0])
print(float_tensor.dtype)

float_tensor = tensor1d.to(torch.float32)
print(float_tensor.dtype)

torch.int64
torch.float32
torch.float32


In [19]:
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor2d)
print(tensor2d.shape)
print(tensor2d.reshape(3, 2))
print(tensor2d.view(3, 2))
print(tensor2d.T)
# matrix multiply
print(tensor2d.matmul(tensor2d.T))
print(tensor2d @ tensor2d.T)


tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])
